<a href="https://colab.research.google.com/github/Pariskimhj/AI_class/blob/master/%EC%B1%97%EB%B4%87%EB%A7%8C%EB%93%A4%EA%B8%B02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
text = pd.read_csv('ChatbotData.csv')
text.head()
print(text.label.value_counts())
text.label.value_counts().plot(kind='bar')
plt.show()
text.loc[text.label==0,].head(3) # 일상다반사 (중립)
text.loc[text.label==1,].head(3) # 이별(부정)
text.loc[text.label==2,].head(3) # 사랑(긍정)
list(text['A'])[:5]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(text['A']))
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)
print(tokenizer.word_index)
sequences = list()
for line in text['A']: # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    sequences.append(encoded)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))
print(list(text['A'])[:5])
print(sequences[:5])
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
max_len = max(len(l) for l in sequences) 
print('샘플의 최대 길이 : {}'.format(max_len))
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)
sequences = np.array(sequences)
X_chatbot = sequences
y_chatbot = text.label
# y confirmation
unique_y = np.unique(y_chatbot)[-1] + 1
print('전체 y의 개수:', len(y_chatbot))
print('고유값 y의 개수:', unique_y)
# 고유값 y각각의 원핫벡터 변경, 총 13399 자리수 맞춤
y_one = to_categorical(y_chatbot, num_classes=unique_y)
print("one_hot_vector y:", y_one)
embedding_dim = 16
hidden_units = 16
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(unique_y, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_chatbot, y_one, epochs=10, validati
embedding_dim = 64
hidden_units = 64

model_lstm = Sequential()
model_lstm.add(Embedding(vocab_size, embedding_dim))
model_lstm.add(LSTM(hidden_units))
model_lstm.add(Dense(unique_y, activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.fit(X_chatbot, y_one, epochs=10, verbose=1, validation_split=0.2)
def sentiment_analysis(model, tokenizer, current_word): # 모델, 토크나이저, 현재 단어
  # 현재 단어에 대한 정수 인코딩과 패딩
  encoded = tokenizer.texts_to_sequences([current_word])[0]
  encoded = pad_sequences([encoded], maxlen=5, padding='pre')
  # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
  result = model.predict(encoded, verbose=0)
  result = np.argmax(result, axis=1)
  if result == [0]:
    word_result = '감성분석: 중립'
  elif result == [1]:
    word_result = '감성분석: 긍정'
  else:
    word_result = '감성분석: 부정' 
  return word_result
sentiment_analysis(model, tokenizer, '하루가 또 가네요')
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = currentword
    sentence = ''

    # n번 반복
    for  in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence
new_text = sentence_generation(model_lstm_ga, tokenizer_ga, '너는', 5)
print('생성단어: {}'.format(new_text))
print(sentiment_analysis(model_lstm_ga, tokenizer_ga, new_text))